In [ ]:
!pip install -U geocoder
!pip install -U geopy
!pip install -U googlemaps
!pip install -U folium

import pandas as pd
import numpy as np

#from bs4 import BeautifulSoup

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import googlemaps
from datetime import datetime
#from geopy.geocoders import GeoNames
#from geopy.geocoders import GoogleV3
#from geopy.geocoders import Nominatim
import requests
import geopy
geopy.geocoders.options.default_user_agent = "my-application"

import time
import folium

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
!wget --cookies=on --save-cookies cookies.txt --keep-session-cookies --post-data 'user=EMAIL&password=PASSWORD' 'https://www.ctt.pt/fecas/login?service=https%3A%2F%2Fwww.ctt.pt%2Ffemgu%2Fj_spring_cas_security_check.jspx'
!wget --cookies=on --load-cookies cookies.txt --keep-session-cookies 'https://www.ctt.pt/feapl_2/app/restricted/postalCodeSearch/postalCodeDownloadFiles!downloadPostalCodeFile.jspx' -O PostCodeFull.Pt.zip

In [ ]:
!rm -f login*
!ls -la

In [ ]:
#!unzip -x todos_cp.zip

In [ ]:
!cat cookies.txt

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
dir(cos)

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [ ]:
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')

In [ ]:
download_file_cos(credentials,'concelhos.txt','concelhos.txt')
download_file_cos(credentials,'distritos.txt','distritos.txt')
download_file_cos(credentials,'todos_cp.txt','todos_cp.txt')
download_file_cos(credentials,'location_geometry.csv','location_geometry.csv')

In [ ]:
!ls -la

In [ ]:
df_DD=pd.read_csv('distritos.txt',sep=';',encoding='iso8859_15')
df_DD

In [ ]:
df_CC=pd.read_csv('concelhos.txt',sep=';',encoding='iso8859_15')
df_CC=df_CC[df_CC['DD']==8]
df_CC

In [ ]:
#columns_1={'DD','CC','LLLL','LOCALIDADE','ART_COD','ART_TIPO','PRI_PREP','ART_TITULO','SEG_PREP','ART_DESIG','ART_LOCAL','TROÇO','PORTA','CLIENTE','CP4','CP3','CPALF'}
df_PostalCode=pd.read_csv('todos_cp.txt',sep=';',encoding='iso8859_15')
#df_PostalCode=pd.read_csv('todos_cp.txt',sep=';',encoding='latin-1',header=None,names=columns_1,usecols=columns_1)[columns_1]
df_PostalCode.shape

In [ ]:
df_PostalCode[df_PostalCode['CPALF']=='LAGOS'].head()

In [ ]:
df_PostalCode=df_PostalCode[['DD','CC','LLLL','LOCALIDADE','CP4','CP3','CPALF']]
df_PostalCode.head()

In [ ]:
#new_df=pd.DataFrame(df_PostalCode[df_PostalCode['DD']==8].groupby(['CPALF','CP4'])['CP3'].count())
#new_df=new_df.reset_index()[['CPALF','CP4']]
#new_df.head()

new_df=pd.DataFrame(df_PostalCode[df_PostalCode['DD']==8].groupby(['CPALF'])['CP4'].count())
new_df=new_df.reset_index()[['CPALF']]
new_df.head()

In [ ]:
new_df.size

In [ ]:
#new_df[['CPALF','CP4']]

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
address='Silves, Algarve, Portugal'
#address='Silves, Algarve, 8300, Portugal'
#location = gmaps.geocode(components={"city": "Silves", "country": "PT"})
#location = gmaps.geocode(components={"city": "Silves", "country": "PT", "postal_code": "8300"})
location = gmaps.geocode(address)
print(location)
result=json_normalize(location)
print(result['geometry.location.lat'][0])
print(result['geometry.location.lng'][0])
print(type(result))
#dir(location)
#latitude = location.latitude
#print(latitude)
#longitude = location.longitude
#print(longitude)

In [ ]:
for ind in new_df.index:
    address='{}, Algarve, Portugal'.format(new_df['CPALF'][ind])
    print(address)

In [ ]:
all_locations=pd.DataFrame()
for ind in new_df.index:
    address='{}, Algarve, Portugal'.format(new_df['CPALF'][ind])
    try:
        gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
        location = gmaps.geocode(address)
    except geopy.exc.GeocoderUnavailable:
        print('GeocoderUnavailable')
        print(name)
    except urllib.error.URLError:
        print('URLError')
        print(name)
    except OSError:
        print('OSError')
        print(name)


    if all_locations.empty:
        all_locations=pd.DataFrame(json_normalize(location))
    else:
        all_locations=all_locations.append(json_normalize(location),sort=False)

#        print(json_normalize(location))
#    print(type(json_normalize(location)))
#    if not location.empty():
#    all_locations.append(json_normalize(location))
#print(len(all_locations))
print(all_locations.shape)

In [ ]:
#all_locations = list(filter(all_locations.empty, all_locations))
all_locations=all_locations.reset_index()
all_locations.head()

In [ ]:
all_locations.dtypes

In [ ]:
location_geometry=all_locations[['geometry.location.lat','geometry.location.lng']]
location_geometry.head()

In [ ]:
location_geometry.to_csv('location_geometry.csv',index=False)

In [ ]:
upload_file_cos(credentials,'location_geometry.csv','location_geometry.csv')

In [ ]:
location_geometry=pd.read_csv('location_geometry.csv',sep=',',encoding='iso8859_15')
location_geometry

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
Resort +
4bf58dd8d48988d12f951735

Vacation Rental - apartamentos
56aa371be4b08b9a8d5734e1

Hotel Pool - just pool
4bf58dd8d48988d132951735

Bed & Breakfast - pensao
4bf58dd8d48988d1f8931735

Boarding House
4f4530a74b9074f6e4fb0100

Inn - not found
5bae9231bedf3950379f89cb

Hostel -
4bf58dd8d48988d1ee931735

Motel - apartamentos
4bf58dd8d48988d1fb931735

Nightlife Spot
4d4b7105d754a06376d81259

In [ ]:
category='4bf58dd8d48988d12f951735' # Resort
#category='4bf58dd8d48988d1fa931735' # Hotel
#category='5bae9231bedf3950379f89cb' # Inn
#category='4f4530a74b9074f6e4fb0100' # Boarding House

latitude=37.089072
longitude=-8.247880
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId={}'.format(
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, 
CLIENT_SECRET,
VERSION,
latitude,
longitude,
1500,
50,
category)
results = json_normalize(requests.get(url).json())
print('Done')

In [ ]:
results.head()

In [ ]:
aa=pd.DataFrame(results['response.venues'][0])
#aa
bb=pd.DataFrame(aa['location'])
bb

In [ ]:
for ind in bb.index:
    zz = bb['location'][ind]
    xx=pd.DataFrame(zz['labeledLatLngs'])
    print(xx['lat'][0],xx['lng'][0])

In [ ]:
#category='4d4b7105d754a06374d81259' # Food
category='52e81612bcbc57f1066b7a01' # Austrian Restaurant

#37.145662, -8.124255 Algarve
#38.723653, -9.137163 Lisbon
#47.072581, 15.440042 Graz
#50.451301, 30.515182 Kiev

latitude='50.451301'
longitude='30.515182'
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId={}'.format(
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, 
CLIENT_SECRET,
VERSION,
latitude,
longitude,
30000,
50,
category)
results = json_normalize(requests.get(url).json())
print('Done')

In [ ]:
results.head()

In [ ]:
aa=pd.DataFrame(results['response.venues'][0])
aa.head()

In [ ]:
latitude=37.089072
longitude=-8.247880
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&categoryId=4bf58dd8d48988d1fa931735'.format(
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, 
CLIENT_SECRET,
VERSION,
latitude,
longitude,
999,
9)
results = json_normalize(requests.get(url).json())
print(results)
try:
    if not results.empty:
        response_groups=results['response.groups'][0]
        #print(type(response_groups))
        print('response_groups')
        print(response_groups)
        if response_groups:
            items=pd.DataFrame(response_groups)['items'][0]
            #print(type(items))
            print('items')
            print(items)
            if items:
                venue=pd.DataFrame(items)['venue'][0]
                #print(type(venue))
                print('venue')
                print(venue)
                if venue:
                    nearby_venues=pd.DataFrame.from_dict(venue,orient='index').T
                    #print(type(nearby_venues))
                    print('nearby_venues')
                    print(nearby_venues)

except KeyError:
    print('error')

#b=results['response.groups'][0]
#c=pd.DataFrame((b))
#d=c['items'][0]
#e=pd.DataFrame(d)['venue'][0]
#name=e['name']
#print(name)
#latitude=e['location']['lat']
#print(latitude)

In [ ]:
print(type(results))
print(type(b))
print(type(c))
print(type(d))
print(type(e))
print(type(name))

In [ ]:
nearby_venues=pd.DataFrame(pd.DataFrame(((results)['response.groups'][0]))['items'][0])['venue'][0]
a=pd.DataFrame.from_dict(nearby_venues,orient='index').T
#a=pd.DataFrame.from_dict(nearby_venues,columns=['name','location'],orient='index')
#nearby_venues=pd.DataFrame.from_dict(pd.DataFrame(pd.DataFrame(((results)['response.groups'][0]))['items'][0])['venue'][0].items()).T
a

In [ ]:
items=pd.DataFrame(pd.DataFrame((results['response.groups'][0]))['items'][0])
items['venue'][0]

In [ ]:
venue=pd.DataFrame(pd.DataFrame(((results)['response.groups'][0]))['items'][0])['venue'][0]
name=venue['name']
print(name)
latitude=venue['location']['lat']
print(latitude)

In [ ]:
all_results=pd.DataFrame()
latitude_sum=0
longitude_sum=0
for ind in all_locations.index:
#for ind in new_df.index:

#    print(ind)
    latitude = all_locations['geometry.location.lat'][ind]
    longitude = all_locations['geometry.location.lng'][ind]
#    print(latitude)
#    print(longitude)
    latitude_sum=latitude_sum+latitude
    longitude_sum=longitude_sum+longitude
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    999,
    5)
#    print(url)

    results = json_normalize(requests.get(url).json())
    print(type(results))
    print('results')
#    id not results.empty and not results['response.groups'][0] and not pd.DataFrame(pd.DataFrame(results['response.groups'][0])['items'][0]).empty and not 
    try:
        if not results.empty:
            response_groups=results['response.groups'][0]
            #print(type(response_groups))
            print('response_groups')
            if response_groups:
                items=pd.DataFrame(response_groups)['items'][0]
                #print(type(items))
                print('items')
                if items:
                    venue=pd.DataFrame(items)['venue'][0]
                    #print(type(venue))
                    print('venue')
                    if venue:
                        nearby_venues=pd.DataFrame.from_dict(venue,orient='index').T
                        #print(type(nearby_venues))
                        print('nearby_venues')
                        print(nearby_venues)
                        if not nearby_venues.empty:
                            if all_results.empty:
                                all_results=pd.DataFrame(nearby_venues)
                            else:
                                all_results=all_results.append(nearby_venues,sort=False)


    except KeyError:
        print('error')

print(len(all_results))
print(latitude_sum)
print(longitude_sum)
print(latitude_sum/len(all_locations))
print(longitude_sum/len(all_locations))

In [ ]:
all_results.head()

In [ ]:
# create map using latitude and longitude values
map_Algarve = folium.Map(location=[latitude_sum/len(all_locations), longitude_sum/len(all_locations)], zoom_start=10)

for lat, lng, venue in zip(all_results['venue.location.lat'],
                                           all_results['venue.location.lng'],
                                           all_results['venue.name']):
    label = 'Venue: {}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Algarve)  
    
map_Algarve

In [ ]:
all_venues=pd.DataFrame()

In [ ]:
for i in range(len(all_results)):
#    print(i)
    venues=all_results[i]['response']['groups'][0]['items']
    nearby_venues = json_normalize(venues)
    all_venues.append(nearby_venues)

print(all_venues.size)

In [ ]:
venues=all_results[0]
#venues=all_results[2]['response']['groups'][0]['items']
#print(type(all_results[15]))
#print(type(all_results[15]['response']))
#print(type(all_results[15]['response']['groups']))
#print(type(all_results[15]['response']['groups'][0]))
#print(type(venues))
nearby_venues = json_normalize(venues)
#((nearby_venues))
#nearby_venues
all_venues.append(nearby_venues)
all_venues
#print(all_venues.size)

In [ ]:
a=all_results[0].get('response')
a.get('groups')

In [ ]:
# Convert the HTML response into a BeautifulSoup Object
soup = BeautifulSoup(all_results[9].content, 'html.parser')

# Use the BeautifulSoup find_all method to extract each top site venue details.
top_venues = soup.find_all('div', class_='venueDetails')

In [ ]:
dir(all_results[9])

In [ ]:
address = 'Lagos (Algarve), Portugal'
geolocator = Nominatim()
location = geolocator.geocode(address)
print((location))
latitude = location.latitude
longitude = location.longitude

print(latitude)
print(longitude)

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    1000,
    100)
url

In [ ]:
results = requests.get(url).json()

In [ ]:
dir(results)

In [ ]:
results.items()

In [ ]:
# Convert the HTML response into a BeautifulSoup Object
soup = BeautifulSoup(results.content, 'html.parser')

# Use the BeautifulSoup find_all method to extract each top site venue details.
top_venues = soup.find_all('div', class_='venueDetails')